# Librerías

In [17]:
''' 
    * Autor: Octavio Augusto Alemán Esparza
    * Fecha: 19.11.2023
    * Título: models_use.ipynb
'''

import pandas as pd
import joblib

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# Cargado de los modelos

Clasificación

In [18]:
classification_model = joblib.load('./model_saves/classification_model.joblib')

Regresión

In [19]:
regression_model = joblib.load('./model_saves/regression_model.joblib')

# Cargado de los datos de prueba

In [20]:
df = pd.read_csv('./data/data_for_model.csv')
df = df.sample(10)
df

,id,ultima ubicacion,dias por posicion,eventual,area empresarial,hijos,edad ingreso,sindicato,posiciones totales,liderazgo,antiguedad,genero,posiciones hasta liderazgo,movimientos reales,dias hasta liderazgo,edad salida,clase
58081,15366793,Liverpool Oaxaca Plaza Bella,80,True,Liverpool,0,22,False,2,False,0.221766,Mujer,-1,1,-1,22,Baja
84638,14903013,Liverpool Guanajuato,171,False,Liverpool,3,51,False,7,False,2.839151,Mujer,-1,3,-1,54,Baja
4223,70115813,Suburbia Símbolos Patrios,177,False,Suburbia,0,22,False,4,False,1.464750,Hombre,-1,3,-1,23,Baja
102579,70617872,Suburbia Tijuana,25,False,Suburbia,0,52,False,3,False,0.142368,Mujer,-1,2,-1,53,Baja
37902,15080927,CeDis Regional León,45,False,CeDis,3,30,False,4,False,0.383299,Hombre,-1,2,-1,30,Baja
66547,14724301,CeDis Regional Poza Rica,188,False,CeDis,2,37,False,6,False,2.592745,Hombre,-1,2,-1,40,Baja
122863,16178205,Liverpool Toreo,1,False,Liverpool,0,21,False,1,False,0.470910,Hombre,-1,1,-1,21,Activo
123464,70723846,Suburbia Fracc Veracruz,27,False,Suburbia,0,28,False,2,False,0.125941,Hombre,-1,1,-1,28,Activo
126487,16153613,Liverpool Tepeyac,1,False,Liverpool,0,33,False,1,False,0.525667,Mujer,-1,1,-1,34,Activo
106539,70606413,Suburbia Lago de Guadalupe,62,False,Suburbia,0,23,False,3,False,0.344969,Hombre,-1,2,-1,24,Baja


# Uso de los modelos

Función para conversión de columnas categóricas

In [21]:
# Conversión de columnas categóricas a numericas
def dataToNumeric(df):

    df_categoric = df.select_dtypes(exclude= ['number'])

    label_encoder = LabelEncoder()
    for col in range (0, len(df_categoric.columns)):
        df[df_categoric.columns[col]] = label_encoder.fit_transform(df[df_categoric.columns[col]])

    return df

Función de etiquetado

In [22]:
def identify_risk(df):
    
    model = classification_model
            
    aux = df.copy()
    aux.drop(['genero', 'sindicato', 'edad ingreso', 'edad salida', 'id'], axis = 1, inplace = True)
    aux = dataToNumeric(aux)

    # Normalización
    n_colsX = aux.shape[1]-1
    X = aux.iloc[:,0:n_colsX]
    rescaledX = StandardScaler().fit_transform(X)
    newX    = pd.DataFrame(data=rescaledX,columns=X.columns)

    # Utilizar el modelo para predecir probabilidades de la clase positiva
    Y_probabilities = model.predict_proba(newX)[:, 1]

    # Definir umbrales de riesgo
    low_threshold = 0.3
    medium_threshold = 0.5

    # Asignar niveles de riesgo
    risk_levels = []

    for score in Y_probabilities:
        if score < low_threshold:
            risk_levels.append("Riesgo Bajo")
        elif low_threshold <= score < medium_threshold:
            risk_levels.append("Riesgo Medio")
        else:
            risk_levels.append("Riesgo Alto")

    # Agregar los niveles de riesgo al DataFrame
    df['Nivel de Riesgo'] = risk_levels

    return df

Función de tiempo

In [23]:
def identify_service(df):

    model = regression_model

    # Colocar antiguedad como última columna
    class_column = df['antiguedad']
    df = df.drop('antiguedad', axis=1)
    df['antiguedad'] = class_column

    aux = df.copy()

    # Eliminación de columnas no usadas
    aux.drop(['clase', 'id', 'genero', 'edad salida', 'edad ingreso', 'eventual', 'sindicato'], axis = 1, inplace = True)

    # Conversión de datos categoricos
    n_colsX = aux.shape[1] - 1
    X = aux.iloc[:, 0:n_colsX]
    X = dataToNumeric(X)

    # Normalization
    rescaledX = StandardScaler().fit_transform(X)
    newX = pd.DataFrame(data=rescaledX, columns=X.columns)

    # Use the model to predict
    y_pred = model.predict(newX)

    # Add the predicted values to the DataFrame
    df['Antigüedad Pronosticada'] = y_pred
    df['Antigüedad Pronosticada'] = df['Antigüedad Pronosticada'].astype(float)

    # Conversión a meses
    df['Antigüedad Pronosticada'] = ((df['Antigüedad Pronosticada'] * 365.25) / 30)
    df['antiguedad'] = ((df['antiguedad'] * 365.25) / 30)

    # Redondear valores
    df['Antigüedad Pronosticada'] =  df['Antigüedad Pronosticada'].astype(int)
    df['antiguedad'] =  df['antiguedad'].astype(int)

    return df

Función conjunta

In [24]:
def use_models(df):

    aux = df.copy()

    aux = identify_risk(aux)

    aux = identify_service(aux)

    return aux

use_models(df)

,id,ultima ubicacion,dias por posicion,eventual,area empresarial,hijos,edad ingreso,sindicato,posiciones totales,liderazgo,genero,posiciones hasta liderazgo,movimientos reales,dias hasta liderazgo,edad salida,clase,Nivel de Riesgo,antiguedad,Antigüedad Pronosticada
58081,15366793,Liverpool Oaxaca Plaza Bella,80,True,Liverpool,0,22,False,2,False,Mujer,-1,1,-1,22,Baja,Riesgo Alto,2,4
84638,14903013,Liverpool Guanajuato,171,False,Liverpool,3,51,False,7,False,Mujer,-1,3,-1,54,Baja,Riesgo Alto,34,141
4223,70115813,Suburbia Símbolos Patrios,177,False,Suburbia,0,22,False,4,False,Hombre,-1,3,-1,23,Baja,Riesgo Bajo,17,68
102579,70617872,Suburbia Tijuana,25,False,Suburbia,0,52,False,3,False,Mujer,-1,2,-1,53,Baja,Riesgo Alto,1,2
37902,15080927,CeDis Regional León,45,False,CeDis,3,30,False,4,False,Hombre,-1,2,-1,30,Baja,Riesgo Alto,4,7
66547,14724301,CeDis Regional Poza Rica,188,False,CeDis,2,37,False,6,False,Hombre,-1,2,-1,40,Baja,Riesgo Alto,31,128
122863,16178205,Liverpool Toreo,1,False,Liverpool,0,21,False,1,False,Hombre,-1,1,-1,21,Activo,Riesgo Medio,5,1
123464,70723846,Suburbia Fracc Veracruz,27,False,Suburbia,0,28,False,2,False,Hombre,-1,1,-1,28,Activo,Riesgo Alto,1,0
126487,16153613,Liverpool Tepeyac,1,False,Liverpool,0,33,False,1,False,Mujer,-1,1,-1,34,Activo,Riesgo Bajo,6,1
106539,70606413,Suburbia Lago de Guadalupe,62,False,Suburbia,0,23,False,3,False,Hombre,-1,2,-1,24,Baja,Riesgo Alto,4,9
